<a href="https://colab.research.google.com/github/MohsenJadidi/Automatic-Modulation-Classification-AMC/blob/master/AMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Importing the dataset***

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
import pickle
fileName = 'RML2016.10a_dict.pkl'
with open("/content/drive/My Drive/Colab Notebooks/"+fileName,'rb') as f:
  data = pickle.load(f,encoding='bytes')

In [0]:
import numpy as np
X = []
labels = [] # label each example by a pair (modulation type, snr)
total_examples = 0
analog = [b'AM-DSB', b'AM-SSB', b'WBFM']

for mod_type, snr in data.keys():
    if snr >=0 and (mod_type not in analog):      
        current_matrix = data[(mod_type, snr)]        
        total_examples += current_matrix.shape[0]
        for i in range(current_matrix.shape[0]):
            X.append(current_matrix[i])
            labels.append((str(mod_type, 'ascii'), snr)) # mod_type is of type bytes
    
X = np.array(X)         # First row is QPSK snr=2, seconde is PAM4 snr=8 , ...
labels = np.array(labels)

y = labels[:,0]

print(f'loaded {total_examples} signal vectors into X{X.shape} and their corresponding'
      f' labels into labels{labels.shape}')  
# print(np.unique(labels[:,0]))


In [0]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
onehotencoder = OneHotEncoder()
y = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()

In [0]:
snrList = [str(2*i) for i in range(10)] 
snr = snrList[9]
numberOfEachExamples = 1000

In [0]:
X = X.reshape([X.shape[0],256])
output = [[labels[i*numberOfEachExamples, 0],y[i*numberOfEachExamples]] for i in range(int(X.shape[0]/numberOfEachExamples))]
output = dict(output)

In [0]:
X_snr = []
y_snr = []

for i in range(X.shape[0]):
    if labels[i,1] == snr:
        X_snr.append(X[i])
        y_snr.append(y[i])

X_snr = np.array(X_snr)
y_snr = np.array(y_snr)  

In [0]:
###### Splitting the dataset into the Training set and Test set ######
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_snr, y_snr, test_size = 0.2, random_state = 0)
# The below line better for Cross_val part
# X_train, X_test, y_train, y_test = train_test_split(X_snr, y_snr, test_size = 1, random_state = 0)


In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

***Making model***

In [0]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 132, kernel_initializer = 'uniform', activation = 'relu', input_dim = 256))
dout = 0.1
classifier.add(Dropout(rate = dout))

# Adding the second hidden layer
classifier.add(Dense(units = 132, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(rate = dout))

# Adding the output layer
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [22]:
batch = 10
epoch = 25
classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/25
6400/6400 [==============================] - 1s 214us/step - loss: 1.7857 - acc: 0.2834
Epoch 2/25
6400/6400 [==============================] - 1s 167us/step - loss: 1.4975 - acc: 0.3808
Epoch 3/25
6400/6400 [==============================] - 1s 169us/step - loss: 1.3860 - acc: 0.4317
Epoch 4/25
6400/6400 [==============================] - 1s 169us/step - loss: 1.3141 - acc: 0.4464
Epoch 5/25
6400/6400 [==============================] - 1s 169us/step - loss: 1.2572 - acc: 0.4772
Epoch 6/25
6400/6400 [==============================] - 1s 170us/step - loss: 1.2254 - acc: 0.4948
Epoch 7/25
6400/6400 [==============================] - 1s 165us/step - loss: 1.1629 - acc: 0.5128
Epoch 8/25
6400/6400 [==============================] - 1s 172us/step - loss: 1.1409 - acc: 0.5309
Epoch 9/25
6400/6400 [==============================] - 1s 167us/step - loss: 1.0719 - acc: 0.5588
Epoch 10/25
6400/6400 [==============================] - 1s 167us/step - loss: 1.0415 - acc: 0.5681
Epoch 11/